# Gather Data

In [60]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#for api calls
import requests
import json
import time

#personal api and aws keys
import config

import query_helper
# import websrcape_wiki_table

## Create climbs db

In [ ]:
query_helper.create_db('climbs')

## Create route table

In [ ]:
route_table = """
CREATE TABLE routes (
    id int NOT NULL,
    name varchar(100),
    type varchar(30),
    rating varchar(20),
    stars float,
    starVotes int,
    pitches int,
    location varchar(255),
    url varchar(255),
    longitude float,
    latitude float,
    PRIMARY KEY (id)
    );
"""

In [8]:
data['routes'][0]

{'id': 105798877,
 'name': 'Fingertrip',
 'type': 'Trad',
 'rating': '5.7',
 'stars': 4.4,
 'starVotes': 486,
 'pitches': 4,
 'location': ['California',
  'Tahquitz & Suicide Rocks',
  'Tahquitz Rock',
  '(g) West Face',
  'West Face - Right Side'],
 'url': 'https://www.mountainproject.com/route/105798877/fingertrip',
 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106161601_sqsmall_1494084533.jpg',
 'imgSmall': 'https://cdn-files.apstatic.com/climb/106161601_small_1494084533.jpg',
 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106161601_smallMed_1494084533.jpg',
 'imgMedium': 'https://cdn-files.apstatic.com/climb/106161601_medium_1494084533.jpg',
 'longitude': -117.1803,
 'latitude': 32.7529}

In [ ]:
query_helper.create_table(route_table)

## Use Mountain Project Api

### Get GPS Coordinates

In [44]:
import xml.etree.ElementTree as ET
tree = ET.parse('waypoints (1).xml')
root = tree.getroot()

In [45]:
root.attrib

{'version': '1.1',
 'creator': 'http://www.geoplaner.com',
 '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd'}

In [46]:
for child in root:
    print(child.attrib)

{'lat': '32.72415', 'lon': '-116.97993'}
{'lat': '32.90872', 'lon': '-117.00192'}
{'lat': '33.12049', 'lon': '-117.07061'}
{'lat': '33.00776', 'lon': '-116.48267'}
{'lat': '33.52393', 'lon': '-117.59544'}
{'lat': '34.13403', 'lon': '-116.3363'}
{'lat': '34.08403', 'lon': '-116.28684'}
{'lat': '34.06129', 'lon': '-116.24288'}
{'lat': '34.02831', 'lon': '-116.21678'}
{'lat': '34.02148', 'lon': '-116.17969'}
{'lat': '34.03058', 'lon': '-116.13024'}
{'lat': '34.08289', 'lon': '-116.14535'}
{'lat': '34.08517', 'lon': '-116.18794'}
{'lat': '34.06925', 'lon': '-116.16046'}
{'lat': '34.02831', 'lon': '-116.09452'}
{'lat': '33.99987', 'lon': '-116.13574'}
{'lat': '34.22034', 'lon': '-117.08074'}
{'lat': '33.98394', 'lon': '-117.4956'}
{'lat': '33.968', 'lon': '-118.3418'}
{'lat': '34.38135', 'lon': '-117.92145'}
{'lat': '34.44581', 'lon': '-119.20785'}
{'lat': '34.58152', 'lon': '-119.97712'}
{'lat': '34.64478', 'lon': '-119.69139'}
{'lat': '35.42697', 'lon': '-118.04296'}
{'lat': '35.48064', '

In [47]:
coordinates = []
for child in root:
    coordinates.append(child.attrib)

In [48]:
coordinates

[{'lat': '32.72415', 'lon': '-116.97993'},
 {'lat': '32.90872', 'lon': '-117.00192'},
 {'lat': '33.12049', 'lon': '-117.07061'},
 {'lat': '33.00776', 'lon': '-116.48267'},
 {'lat': '33.52393', 'lon': '-117.59544'},
 {'lat': '34.13403', 'lon': '-116.3363'},
 {'lat': '34.08403', 'lon': '-116.28684'},
 {'lat': '34.06129', 'lon': '-116.24288'},
 {'lat': '34.02831', 'lon': '-116.21678'},
 {'lat': '34.02148', 'lon': '-116.17969'},
 {'lat': '34.03058', 'lon': '-116.13024'},
 {'lat': '34.08289', 'lon': '-116.14535'},
 {'lat': '34.08517', 'lon': '-116.18794'},
 {'lat': '34.06925', 'lon': '-116.16046'},
 {'lat': '34.02831', 'lon': '-116.09452'},
 {'lat': '33.99987', 'lon': '-116.13574'},
 {'lat': '34.22034', 'lon': '-117.08074'},
 {'lat': '33.98394', 'lon': '-117.4956'},
 {'lat': '33.968', 'lon': '-118.3418'},
 {'lat': '34.38135', 'lon': '-117.92145'},
 {'lat': '34.44581', 'lon': '-119.20785'},
 {'lat': '34.58152', 'lon': '-119.97712'},
 {'lat': '34.64478', 'lon': '-119.69139'},
 {'lat': '35.426

In [56]:
for i,coord in enumerate(coordinates):
    print(i,coord['lat'], coord['lon'])

0 32.72415 -116.97993
1 32.90872 -117.00192
2 33.12049 -117.07061
3 33.00776 -116.48267
4 33.52393 -117.59544
5 34.13403 -116.3363
6 34.08403 -116.28684
7 34.06129 -116.24288
8 34.02831 -116.21678
9 34.02148 -116.17969
10 34.03058 -116.13024
11 34.08289 -116.14535
12 34.08517 -116.18794
13 34.06925 -116.16046
14 34.02831 -116.09452
15 33.99987 -116.13574
16 34.22034 -117.08074
17 33.98394 -117.4956
18 33.968 -118.3418
19 34.38135 -117.92145
20 34.44581 -119.20785
21 34.58152 -119.97712
22 34.64478 -119.69139
23 35.42697 -118.04296
24 35.48064 -119.16389
25 35.52533 -120.68045
26 36.27248 -119.40566
27 36.52596 -118.21329
28 35.93668 -120.89595
29 36.61828 -121.59928
30 37.30264 -121.99491
31 40.29436 -123.03421
32 38.9267 -121.00584
33 38.95673 -119.95385
34 38.93751 -119.99507
35 38.91829 -119.95385
36 38.88411 -120.12694
37 38.9802 -120.22035
38 39.14433 -120.22035
39 39.23794 -120.1654
40 39.29745 -120.0143
41 35.80545 -117.48531
42 37.74192 -119.71865
43 37.73758 -119.68019
44 37.7

In [49]:
len(coordinates)

321

### Using GPS Coordinates

In [59]:
def mp_gps_call(url_params, api_key):
# https://www.mountainproject.com/data/get-routes-for-lat-lon?lat=40.03&lon=-105.25&maxDistance=10&minDiff=5.6&maxDiff=5.10&key=112327568-ec557ff70496df7d0a43436ab9df1201
    url = 'https://www.mountainproject.com/data/get-routes-for-lat-lon'
#     headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, params=url_params)
    
    return response.json()

#### Loop over list of GPS coordinates, get routes and insert into db

In [3]:
for i,coord in enumerate(coordinates):
    ##assign new lat long and use api
    api_key = config.api_key
    lat = coord['lat']   #32.904600
    lon = coord['lon']  #-116.818400
    maxResults =500
    maxDistance = 10
    minDiff = '5.0'
    maxDiff = '5.16'

    url_params = {  'lat': str(lat),
                    'lon': str(lon),
                    'maxResults': maxResults,
    #                 'maxDistance' : str(maxDistance),
    #                 'minDiff': str(minDiff),
    #                 'maxDiff': str(maxDiff)
                    'key': api_key
                 }
    # call function and return 5000 route details
    data = mp_gps_call(url_params, api_key)
    
    #check that call successful 
    if data['success']:
        routes = data['routes']
        
        #store route data in mysql
        query_helper.route_to_tup(routes)
        
    #print error message if something goes wrong
    else:
        f'Error!! stopped at {i} iteration {lat},{lon}'
        
    #sleep for a bit to avoid being blocked
    time.sleep(3)


In [51]:
data = mp_gps_call(url_params, api_key)
# print(data)

In [52]:
print(len(data['routes']))
# print(data['routes'])

500


In [57]:
data['success']

1

In [53]:
for i in range(20):
    print(data['routes'][i], '\n')

{'id': 105798877, 'name': 'Fingertrip', 'type': 'Trad', 'rating': '5.7', 'stars': 4.4, 'starVotes': 486, 'pitches': 4, 'location': ['California', 'Tahquitz & Suicide Rocks', 'Tahquitz Rock', '(g) West Face', 'West Face - Right Side'], 'url': 'https://www.mountainproject.com/route/105798877/fingertrip', 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106161601_sqsmall_1494084533.jpg', 'imgSmall': 'https://cdn-files.apstatic.com/climb/106161601_small_1494084533.jpg', 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106161601_smallMed_1494084533.jpg', 'imgMedium': 'https://cdn-files.apstatic.com/climb/106161601_medium_1494084533.jpg', 'longitude': -117.1803, 'latitude': 32.7529} 

{'id': 105793305, 'name': 'Leonids', 'type': 'Sport', 'rating': '5.9', 'stars': 4.6, 'starVotes': 243, 'pitches': 3, 'location': ['California', 'San Diego and Environs', 'South San Diego County', 'El Cajon Mountain', 'The Wedge', 'Left Wall'], 'url': 'https://www.mountainproject.com/route/105793305/leoni

In [54]:
routes = data['routes']

In [55]:
routes[0]

{'id': 105798877,
 'name': 'Fingertrip',
 'type': 'Trad',
 'rating': '5.7',
 'stars': 4.4,
 'starVotes': 486,
 'pitches': 4,
 'location': ['California',
  'Tahquitz & Suicide Rocks',
  'Tahquitz Rock',
  '(g) West Face',
  'West Face - Right Side'],
 'url': 'https://www.mountainproject.com/route/105798877/fingertrip',
 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/106161601_sqsmall_1494084533.jpg',
 'imgSmall': 'https://cdn-files.apstatic.com/climb/106161601_small_1494084533.jpg',
 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/106161601_smallMed_1494084533.jpg',
 'imgMedium': 'https://cdn-files.apstatic.com/climb/106161601_medium_1494084533.jpg',
 'longitude': -117.1803,
 'latitude': 32.7529}

In [19]:
for i, route in enumerate(routes):
    if len(route)!=15:
        print(i, route)
        break

### Store Results in AWS DB

In [ ]:
route_to_tup(routes)